In [ ]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

In [ ]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName('RIK_Lab1')
         .getOrCreate())

sc = spark.sparkContext

### Задаём вариант

In [ ]:
# Вариант 37
id_film = '302'

### Готовим RDD для расчетов

In [ ]:
# structure: "user id" \t "item id" \t "rating" \t "timestamp"
rdd = (sc
       .textFile("/labs/laba01/ml-100k/u.data")
       .map(lambda x: x.split("\t"))
       .cache())

In [ ]:
rdd.take(10)

### Расчет общего рейтинга всех фильмов

In [ ]:
rating_all_film = (rdd
                   .map(lambda x: (x[2], 1))
                   .reduceByKey(lambda acc, val: acc + val)
                   .collect())

rating_all_film.sort()
rating_all_film

### Расчет рейтина фильма по варианту

In [ ]:
rating_one_film = (rdd
                   .filter(lambda x: x[1] == id_film)
                   .map(lambda x: (x[2], 1))
                   .reduceByKey(lambda acc, val: acc + val)
                   .collect())

rating_one_film.sort()
rating_one_film

### Собираем результат

In [ ]:
result = {
   "hist_film": [x[1] for x in rating_one_film],
   "hist_all": [x[1] for x in rating_all_film]
}
result

### Сохраняем lab01.json

In [ ]:
import json

with open('lab01.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(result))

# Вариант с расчетом рейтингов для всех фильмов

In [ ]:
# dict нулевого рейтига: {'1':0, ... '5': 0}
zero_rating = {str(k): 0 for k in range(1, 6)}

def sum_two_rating(rating1, rating2):
    return {k: rating1[k] + rating2[k] for k in [str(k_int) for k_int in range(1, 6)]}

def sum_list_rating(ratings):
    result = zero_rating
    for rating in ratings:
        result = sum_two_rating(result, rating)
    return result

In [ ]:
rdd_full_raging = (sc
       .textFile("/labs/laba01/ml-100k/u.data")
       .map(lambda x: x.split("\t"))
       .map(lambda x: (x[1], {**zero_rating, **{x[2]: 1}}))
       .groupByKey()
       .map(lambda x: (x[0], sum_list_rating(x[1]))))

In [ ]:
# Рейтинги для всех фильмов
rdd_full_raging.take(10)

In [ ]:
# Рейтинг фильма по заданию
rdd_full_raging.filter(lambda x: x[0] == id_film).collect()

In [ ]:
# Общий рейтинг всех фильмов
rdd_full_raging.map(lambda x: x[1]).reduce(sum_two_rating)

In [ ]:
spark.stop()